In [2]:
import glob

ROOT_PATH = "/"
DATA_PATH = "./input"
DB_FILE = "%s/*.csv" % DATA_PATH # ./input/*.csv

files = glob.glob(DB_FILE)

print(files)

['./input/hpg_reserve.csv', './input/date_info.csv', './input/air_visit_data.csv', './input/hpg_store_info.csv', './input/air_reserve.csv', './input/air_store_info.csv', './input/sample_submission.csv']


In [45]:
import pandas as pd
df = {}

for filename in files:
    name = filename[8:] # get me only the filename.csv without ./input/
    df[name.replace('.csv', '')] = pd.read_csv(filename) 

print(df.keys())
#print(df.values())

dict_keys(['hpg_reserve', 'date_info', 'air_visit_data', 'hpg_store_info', 'air_reserve', 'air_store_info', 'sample_submission'])


## Data Exploring

Working with the data with the help of libery of pandas.
First step we would like to expolre our Data set.

Comparison with SQL

In [50]:
import numpy as np

# **** In SQL **** #
# SELECT *
# FROM air_visit_data
# LIMIT 6;

#  air_store_id  visit_date  visitors

#
#print(df['air_visit_data'].head())

air_visit = df['air_visit_data']

#print(air_visit.shape)

air_visit.index = pd.to_datetime(air_visit['visit_date']) # the rows are now by visit_date

#print(df['air_visit_data'].head())

#

air_visit = air_visit.groupby('air_store_id').apply(lambda g: g['visitors'].resample('1d').sum()).reset_index()
air_visit['visit_date'] = air_visit['visit_date'].dt.strftime('%Y-%m-%d')
air_visit['was_nil'] = air_visit['visitors'].isnull()
air_visit['visitors'].fillna(0, inplace=True)
air_visit.head()





,air_store_id,visit_date,visitors,was_nil
0,air_00a91d42b08b08d9,2016-07-01,35,False
1,air_00a91d42b08b08d9,2016-07-02,9,False
2,air_00a91d42b08b08d9,2016-07-03,0,False
3,air_00a91d42b08b08d9,2016-07-04,20,False
4,air_00a91d42b08b08d9,2016-07-05,25,False


In [22]:
import numpy as np

# **** In SQL **** #
# SELECT air_store_id ,air_genre_name , longitude
# FROM air_store_info
# LIMIT 6;

#df['air_store_info'][['air_store_id' ,'air_genre_name' , 'longitude']].head(6)

# Check the length and information of 'air_store_info'
print(len(df['air_store_info']))

# ************************************************ WHERE ***************************************************** #
# SELECT *
# FROM air_store_info
# WHERE air_genre_name = 'Okonomiyaki/Monja/Teppanyaki'
# LIMIT 6;

#df['air_store_info'][df['air_store_info']['air_genre_name'] == 'Okonomiyaki/Monja/Teppanyaki'].head(6)

#The above statement is simply passing a Series of 
#True/False objects to the DataFrame, returning all rows with True.

is_okonomiyaki = df['air_store_info']['air_genre_name'] == 'Okonomiyaki/Monja/Teppanyaki'

is_okonomiyaki.value_counts()

# ************************************************************************************************************* # 

829


False    815
True      14
Name: air_genre_name, dtype: int64

In [11]:
# Check the length and information of 'air_store_info'
print(len(df['air_store_info']))
#print(df['air_store_info']['air_store_id'].unique())
print(df['air_store_info']['air_genre_name'].unique())
print(len(df['air_store_info']['air_area_name'].unique()))
df['air_store_info'].head()

829
['Italian/French' 'Dining bar' 'Yakiniku/Korean food' 'Cafe/Sweets'
 'Izakaya' 'Okonomiyaki/Monja/Teppanyaki' 'Bar/Cocktail' 'Japanese food'
 'Creative cuisine' 'Other' 'Western food' 'International cuisine' 'Asian'
 'Karaoke/Party']
103


,air_store_id,air_genre_name,air_area_name,latitude,longitude
0,air_0f0cdeee6c9bf3d7,Italian/French,Hyōgo-ken Kōbe-shi Kumoidōri,34.695124,135.197852
1,air_7cc17a324ae5c7dc,Italian/French,Hyōgo-ken Kōbe-shi Kumoidōri,34.695124,135.197852
2,air_fee8dcf4d619598e,Italian/French,Hyōgo-ken Kōbe-shi Kumoidōri,34.695124,135.197852
3,air_a17f0778617c76e2,Italian/French,Hyōgo-ken Kōbe-shi Kumoidōri,34.695124,135.197852
4,air_83db5aff8f50478e,Italian/French,Tōkyō-to Minato-ku Shibakōen,35.658068,139.751599


In [19]:
# Check the length and information of 'hpg_store_info'
print(len(df['hpg_store_info']))
df['hpg_store_info'].head()

4690


,hpg_store_id,hpg_genre_name,hpg_area_name,latitude,longitude
0,hpg_6622b62385aec8bf,Japanese style,Tōkyō-to Setagaya-ku Taishidō,35.643675,139.668221
1,hpg_e9e068dd49c5fa00,Japanese style,Tōkyō-to Setagaya-ku Taishidō,35.643675,139.668221
2,hpg_2976f7acb4b3a3bc,Japanese style,Tōkyō-to Setagaya-ku Taishidō,35.643675,139.668221
3,hpg_e51a522e098f024c,Japanese style,Tōkyō-to Setagaya-ku Taishidō,35.643675,139.668221
4,hpg_e3d0e1519894f275,Japanese style,Tōkyō-to Setagaya-ku Taishidō,35.643675,139.668221


In [20]:
# Check the length and information of 'air_reserve'
print(len(df['air_reserve']))
df['air_reserve'].head()

92378


,air_store_id,visit_datetime,reserve_datetime,reserve_visitors
0,air_877f79706adbfb06,2016-01-01 19:00:00,2016-01-01 16:00:00,1
1,air_db4b38ebe7a7ceff,2016-01-01 19:00:00,2016-01-01 19:00:00,3
2,air_db4b38ebe7a7ceff,2016-01-01 19:00:00,2016-01-01 19:00:00,6
3,air_877f79706adbfb06,2016-01-01 20:00:00,2016-01-01 16:00:00,2
4,air_db80363d35f10926,2016-01-01 20:00:00,2016-01-01 01:00:00,5


In [21]:
# Check the length and information of 'hpg_reserve'
print(len(df['hpg_reserve']))
df['hpg_reserve'].head()

2000320


,hpg_store_id,visit_datetime,reserve_datetime,reserve_visitors
0,hpg_c63f6f42e088e50f,2016-01-01 11:00:00,2016-01-01 09:00:00,1
1,hpg_dac72789163a3f47,2016-01-01 13:00:00,2016-01-01 06:00:00,3
2,hpg_c8e24dcf51ca1eb5,2016-01-01 16:00:00,2016-01-01 14:00:00,2
3,hpg_24bb207e5fd49d4a,2016-01-01 17:00:00,2016-01-01 11:00:00,5
4,hpg_25291c542ebb3bc2,2016-01-01 17:00:00,2016-01-01 03:00:00,13


In [9]:
# Check all tables

#for index in df:
#    print(len(df[index]))
#    print(df[index].head())
#    print()


#df['air_reserve'].dtypes
df['air_reserve'].shape
    


(92378, 4)

In [ ]:
#https://www.datacamp.com/community/tutorials/pandas-tutorial-dataframe-python?utm_source=adwords_ppc&utm_campaignid=898687156&utm_adgroupid=48947256715&utm_device=c&utm_keyword=&utm_matchtype=b&utm_network=g&utm_adpostion=1t1&utm_creative=255768618198&utm_targetid=aud-522010995285:dsa-473406573835&utm_loc_interest_ms=&utm_loc_physical_ms=1008006&gclid=Cj0KCQiAj4biBRC-ARIsAA4WaFjCvIyVvtP07ACMo3ijuyMUync2BFdSZbVuAt25bdhnobhJUK1Dd5waAn6FEALw_wcB#question2
